In [2]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../Configuration"

import refinitiv.data as rd
from refinitiv.data.content import symbol_conversion

import time
import math
import pandas as pd
import numpy as np
from datetime import datetime


import mysql.connector as mysql

from queue import Queue
from threading import Thread

import pandas_market_calendars as mcal


import importlib
import warnings

warnings.simplefilter(action='ignore', category=UserWarning)

# My library

import utils.fidatabase as fi

#import utils.variant_callback as vc



In [3]:
rd.open_session()
db = fi.ConnectToDb()

In [4]:

rics_of_interest = fi.getSPYList(rd, pd.Timestamp.today().date())


print(rics_of_interest)

Joined twice: HET.N^A08
Deleting added ref HET.N^A08
Joined twice: ROK.N
Deleting added ref ROK.N
Joined twice: MII.N^F99
Deleting added ref MII.N^F99
Joined twice: FJ.N^K00
Deleting added ref FJ.N^K00
Joined twice: RX.N^C10
Deleting added ref RX.N^C10
Joined twice: MWW.N^K16
Deleting added ref MWW.N^K16
Joined twice: PEG.N
Deleting added ref PEG.N
Joined twice: PARA.OQ
Deleting added ref PARA.OQ
Joined twice: JCI.N
Deleting added ref JCI.N
Not found to remove!:  FTI.N
['CPB.N', 'COST.OQ', 'AXP.N', 'PCAR.OQ', 'CLX.N', 'AAPL.OQ', 'HON.OQ', 'NWL.OQ', 'BMY.N', 'T.N', 'AIG.N', 'TXN.OQ', 'EIX.N', 'SWK.N', 'GLW.N', 'IBM.N', 'MS.N', 'AMGN.OQ', 'UNP.N', 'GE.N', 'BA.N', 'EXC.OQ', 'PH.N', 'NSC.N', 'MCD.N', 'K.N', 'ITW.N', 'PG.N', 'D.N', 'HPQ.N', 'XOM.N', 'BKR.OQ', 'BALL.N', 'ROK.N', 'APD.N', 'CSX.OQ', 'LUV.N', 'LOW.N', 'WMB.N', 'SLB.N', 'CL.N', 'HES.N', 'COP.N', 'MSFT.OQ', 'IP.N', 'ETN.N', 'WFC.N', 'MMM.N', 'TRV.N', 'ECL.N', 'LLY.N', 'CAT.N', 'TAP.N', 'JPM.N', 'BDX.N', 'AVY.N', 'WMT.N', 'GL.N', 

/Users/eugenemesgar/fintech/stockdata/utils/fidatabase.py:742: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if index > index_date:


In [5]:
create_new_from_rdp = False

# Set options for workbook
series_name = "PriceIntrp"
#rdp_series_name = "TR.PriceClose"
#rdp_series_param = "Curn=USD"


In [6]:
output_df = pd.DataFrame()
series_metadata = {}


importlib.reload(fi)

if(create_new_from_rdp):
    end_date = pd.Timestamp.today().date()
    start_date = pd.Timestamp(year=end_date.year-30,month=1,day=1).date()

    output_df = fi.ReadTimeSeriesFromRDP(rd,rdp_series_name,rdp_series_param, start_date, end_date,rics_of_interest)
    output_df = fi.ClearDupes(output_df)

    
    series_metadata["series_name"] = series_name
    series_metadata["rdp_series_name"] = rdp_series_name
    series_metadata["rdp_series_param"] = rdp_series_param
    series_metadata["base_series"] = None
    series_metadata["update_function"] = None
    

    series_metadata["start_date"] = output_df.first_valid_index().date()
    series_metadata["end_date"] = output_df.last_valid_index().date()
    series_metadata["ric_list"] = rics_of_interest

    #fi.WriteSeriesToDB(db, series_metadata, output_df,True)
else:
    result = fi.UpdateSeries(rd,db, series_name)
    series_metadata = fi.GetSeriesMetadata(db, series_name)
    output_df = fi.ReadSeriesFromDB( db, series_name )



No new data to pull


In [7]:
# Find all the nan sequnces
sorted_key, run_hash= fi.FindNanRuns( output_df )


i=0
for val in enumerate(sorted_key):
    i+=1
    
    run_count = val[1][1]
    run_ric = val[1][0]
    if(run_count > 0):
        run_list = run_hash[run_ric]
        print(  "RIC: ", run_ric, " Missing: ", run_count)
        for j in range(0, min(5,len(run_list))):
            print(f"    {run_list[j][2]}: from {run_list[j][0]} to {run_list[j][1]}")

        if(i>30):
            break


In [ ]:
#Interoplate the data out
output_df.interpolate(method='time', inplace=True,limit_direction='both', limit_area='inside')


In [ ]:
sorted_key, run_hash= fi.FindNanRuns( output_df )

if( sorted_key[0][1] > 0 ):
    print("Error: There are still missing values in the data")

In [ ]:
# # Write the interpolated data to the database
importlib.reload(fi)

new_metadata = series_metadata.copy()

new_metadata["series_name"] = series_metadata["series_name"] + "Norm"
new_metadata["base_series"] =  series_metadata["series_name"]
new_metadata["rdp_series_name"] = None
new_metadata['rdp_series_param'] = None
new_metadata["update_function"] = "TransformSeriesToRangeCallback"

fi.UpdateSeriesMetadata(db, new_metadata["series_name"], new_metadata,True)

new_df = fi.ReadSeriesFromDB( db, new_metadata["series_name"] )

# result = fi.WriteSeriesToDB(db, new_metadata, output_df,True)



In [ ]:
importlib.reload(fi)

new_df = fi.ReadSeriesFromDB( db, "PriceIntrpNorm" )


In [ ]:
new_df["MSFT.OQ"].plot(figsize=(20,10))

In [ ]:
importlib.reload(fi)

#
# Truncate data series so we can update
#

if(~create_new_from_rdp):

    cds = db.cursor()
    
    # mdf = fi.GetSeriesMetadata(db, "Price")
    # mdf["end_date"] = pd.Timestamp(year=2023,month=3,day=14).date()

    # sql = f"DELETE FROM ts_Price WHERE ddate > '{mdf['end_date']}';"
    # cds.execute(sql)
    
    # result =fi.UpdateSeriesMetadata(db, "Price", mdf)


    # Update the derived series
    #mdf = fi.GetSeriesMetadata(db, "PriceIntrp")
    #mdf["end_date"] = pd.Timestamp(year=2015,month=3,day=14).date()

    #sql = f"DELETE FROM ts_PriceIntrp WHERE ddate > '{mdf['end_date']}';"
    #cds.execute(sql)

    #result = fi.UpdateSeriesMetadata(db, "PriceIntrp", mdf)

    #db.commit()



    #cds.close()

    #print(result)

In [ ]:
importlib.reload(fi)
fi.UpdateSeries( rd, db, "PriceIntrp" )

In [ ]:
# # Write to database
# def WriteSeriesToDB( db, series_metadata, new_df, new_series=False):

#     if(new_series==False):
#         mdf= fi.GetSeriesMetadata( db, series_metadata["series_name"] )
#         if( len(mdf) == 0) :
#             print("Series does not exist in database.  Please rerun w/ new_series=True.")
#             return False
        
#     series_name = series_metadata["series_name"]
#     rdp_series_name = series_metadata["rdp_series_name"]
#     rdp_series_params = series_metadata["rdp_series_param"]
#     start_date = series_metadata["start_date"]
#     end_date = series_metadata["end_date"]
#     ric_list = ",".join(series_metadata["ric_list"])
    
#     #rdp_series_name = "TR.PriceClose"
#     #rdp_series_params = "Curn=USD"
#     #start_date = new_df.first_valid_index().date()
#     #end_date = new_df.last_valid_index().date()
#     #ric_list = ",".join(rics_of_interest)

#     table_name=""

#     if(new_series==True):
#         table_name = fiutil.CreateNewSeriesTable( db, rics_of_interest, series_name )
#     else:
#         table_name = "ts_" + series_name


#     dbs = db.cursor()

#     header_sql = f"INSERT INTO {table_name}  ( ddate, "
#     for ric in  new_df.columns:
#         col = ric.replace(".", "_") 
#         header_sql = header_sql + col + ", "

#     header_sql = header_sql[:-2] 

#     row_count = len(new_df.index)

#     chunk_size = 90

#     for i in range(0, row_count, chunk_size):
#         sql = header_sql + ") VALUES \n"

#         next_chunk = min( row_count, chunk_size+i)

#         for j in range(i, next_chunk):
#             sql = sql + "('%s', " % (new_df.index[j].date())
#             for ric in  new_df.columns:
#                 val = new_df.at[new_df.index[j], ric]
#                 if( pd.isna(val) or math.isnan(val) ):
#                     sql = sql + "NULL, "
#                 else:
#                     sql = sql + f"{val:.2f}, "

#             sql = sql[:-2] 
#             sql = sql + "),\n"
            

#         sql = sql[:-2] 
#         sql = sql + ";"
        
#         #print(sql)
#         dbs.execute(sql)

#     meta_sql=""
#     if(new_series==True):
#         meta_sql = f"INSERT INTO TimeSeries_MetaData ( series_name, rdp_series_name, rdp_series_parms, start_date, end_date, ric_list ) VALUES "
#         meta_sql = meta_sql + f"('{series_name}', '{rdp_series_name}', '{rdp_series_params}', '{start_date}', '{end_date}', '{ric_list}');"
#     else:
#         meta_sql = f"UPDATE TimeSeries_MetaData SET end_date='{end_date}' WHERE series_name='{series_name}';"
    

#     dbs.execute(meta_sql)
#     db.commit()
#     dbs.close()
#     return True
        
        

In [ ]:
# Read file "S&P500PriceHistory.csv
# The file includes a header row
# Intrepret dates as dates
# The date column is the index

# if(pull_from_refinitiv==False):
#     output_df = pd.read_csv("S&P500PriceHistory.csv", parse_dates=['Date'], index_col='Date')
# else:
#     stamp = str(datetime.now())
#     output_file = f"S&P500PriceHistory-{start_date}-{end_date}-{stamp}.csv"
#     output_df.to_csv(output_file)
#     print("Wrote to", output_file)
    


In [ ]:
stamp = str(datetime.now())
output_file = f"CLEANED S&P500PriceHistory-{start_date}-{end_date}-{stamp}.csv"
new_df.to_csv(output_file)
print("Wrote to", output_file)



In [ ]:
# Plot new_df["MRO.N"] from 1990 to 1995

#interp_d2["CSCO.OQ"].plot(x="Date", ylim=[0,1.5], xlim=[pd.Timestamp(year=1990,day=1,month=1).date(),pd.Timestamp(year=1993,day=1,month=1).date()], y="MRO.N", figsize=(20,10), title="MRO.N")




In [ ]:
interp_df = new_df.copy()




#interp_d2 = interp_df.applymap( lambda x: np.nan if pd.isna(x) else float(x) )
                            
interp_df.interpolate(method='time', inplace=True,limit_direction='both', limit_area='inside')

new_df_old = new_df.copy()
new_df = interp_df


In [ ]:


new_df = new_df_old
#interp_d2["CSCO.OQ"].plot(x="Date", ylim=[0,1.5], xlim=[pd.Timestamp(year=1991,day=1,month=1).date(),pd.Timestamp(year=1991,day=1,month=1).date()], y="MRO.N", figsize=(20,10), title="MRO.N")






In [ ]:
# Commented Out -- Former code to write to database


# index = 0
# col_count = 0



# if(pull_from_refinitiv==True):

#     dbs = db.cursor()
#     db.begin()

#     for col in output_df.columns:
#         print(f"Processing column {col} ({col_count} of {len(output_df.columns)})")


#         col_count = col_count + 1
#         sql = ""
#         to_write = output_df[col].copy()
#         to_write.dropna(inplace=True)

#         first_col = 0
#         last_col = len(to_write)
        
        
#         chunk_size = 10000
        
        
#         for i in range(first_col, last_col, chunk_size):
#             sql = "INSERT INTO TimeSeries ( ric, series, variant, ddate, val ) VALUES " 
    

#             next_chunk = min( last_col, chunk_size+i)
#             for j in range(i, next_chunk):

#                 date_val = str(to_write.index[j].date())
#                 value_val = to_write[j]
#                 #my_data.append( ( col, "Price", "RDP", date_val, value_val) ) 

#                 sql = sql + f'("{col}", "Price", "RDP", "{date_val}", "{value_val:.2f}")'
#                 if( j < (next_chunk-1) ):
#                     sql = sql + ","
#                 else:
#                     sql = sql + ";"

#                 sql = sql + "\n"
        
        
#             ## Execute the sql statement stored in variable sql
#             dbs.execute(sql)
            
#     db.commit()
#     dbs.close()

    
       
       


 


In [ ]:
# # Write to database
# def WriteSeriesToDB( db, series_metadata, new_df, new_series=False):

#     if(new_series==False):
#         mdf= fiutil.GetSeriesMetadata( db, series_metadata["series_name"] )
#         if( len(mdf) == 0) :
#             print("Series does not exist in database.  Please rerun w/ new_series=True.")
#             return False
        
#     series_name = series_metadata["series_name"]
#     rdp_series_name = series_metadata["rdp_series_name"]
#     rdp_series_params = series_metadata["rdp_series_params"]
#     start_date = series_metadata["start_date"]
#     end_date = series_metadata["end_date"]
#     ric_list = ",".join(series_metadata["ric_list"])
    
#     #rdp_series_name = "TR.PriceClose"
#     #rdp_series_params = "Curn=USD"
#     #start_date = new_df.first_valid_index().date()
#     #end_date = new_df.last_valid_index().date()
#     #ric_list = ",".join(rics_of_interest)

#     table_name=""

#     if(new_series==True):
#         table_name = fiutil.CreateNewSeriesTable( db, rics_of_interest, series_name )
#     else:
#         table_name = "ts_" + series_name


#     dbs = db.cursor()

#     header_sql = f"INSERT INTO {table_name}  ( ddate, "
#     for ric in  new_df.columns:
#         col = ric.replace(".", "_") 
#         header_sql = header_sql + col + ", "

#     header_sql = header_sql[:-2] 

#     row_count = len(new_df.index)

#     chunk_size = 90

#     for i in range(0, row_count, chunk_size):
#         sql = header_sql + ") VALUES \n"

#         next_chunk = min( row_count, chunk_size+i)

#         for j in range(i, next_chunk):
#             sql = sql + "('%s', " % (new_df.index[j].date())
#             for ric in  new_df.columns:
#                 val = new_df.at[new_df.index[j], ric]
#                 if( pd.isna(val) or math.isnan(val) ):
#                     sql = sql + "NULL, "
#                 else:
#                     sql = sql + f"{val:.2f}, "

#             sql = sql[:-2] 
#             sql = sql + "),\n"
            

#         sql = sql[:-2] 
#         sql = sql + ";"
        
#         #print(sql)
#         dbs.execute(sql)

#     meta_sql=""
#     if(new_series==True):
#         meta_sql = f"INSERT INTO TimeSeries_MetaData ( series_name, rdp_series_name, rdp_series_parms, start_date, end_date, ric_list ) VALUES "
#         meta_sql = meta_sql + f"('{series_name}', '{rdp_series_name}', '{rdp_series_params}', '{start_date}', '{end_date}', '{ric_list}');"
#     else:
#         meta_sql = f"UPDATE TimeSeries_MetaData SET end_date='{end_date}' WHERE series_name='{series_name}';"
    

#     dbs.execute(meta_sql)
#     db.commit()
#     dbs.close()
#     return True
        


In [ ]:
# Commented Out -- Former code to read from database
#

# start_date=pd.Timestamp(year=2000,day=1,month=1)
# end_date=pd.Timestamp(year=2023,day=1,month=1)


# #Find the valid days in the calendar that there is data for
# valid_days = mcal.get_calendar('NYSE').schedule(start_date=start_date, end_date=end_date).index.to_list()

# # Create an empty dayaframe with the index & columns we want
# read_df = pd.DataFrame(np.nan, columns=rics_of_interest,index=valid_days)

# #Create the SQL statement to query the DB
# sql = "SELECT ric, ddate, val FROM TimeSeries WHERE variant = 'RDP' AND ddate >= '%s' AND ddate <= '%s'" % (start_date, end_date)


# #Read the SQL into a dataframe
# ts = time.perf_counter()
# raw_df = pd.read_sql(sql, db, parse_dates=True, coerce_float=True)
# te = time.perf_counter()

# print(f"Read {len(raw_df)} rows in {te-ts:0.4f} seconds")

# ts = time.perf_counter()



# for row in raw_df.itertuples():
#     read_df.at[ pd.Timestamp(row.ddate), row.ric ] = row.val


# te = time.perf_counter()
# print(f"Processed SQL to DF rows in {te-ts:0.4f} seconds")

# read_df.head()
